# Seoul Bike Sharing — EDA + Limpieza (Modified) y Comparación vs Original (Referencia) + Baseline ML

**Reglas**
- Trabajamos y modelamos con **`seoul_bike_sharing_modified.csv`** (principal).
- **`seoul_bike_sharing_original.csv`** es **solo referencia** para validar limpieza/distribución.
- Ambos CSV en el **mismo directorio** que este notebook en Drive. El notebook puede *auto-descubrir* si no das la ruta.


## 1) Setup e imports

In [ ]:
#@title Setup e imports
import os, sys, warnings, json, hashlib, textwrap, math
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
print('Versions -> numpy', np.__version__, '| pandas', pd.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2) Ubicación de archivos

In [ ]:
#@title 🔎 Diagnóstico: buscar el archivo "modified" en todo tu Drive
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

bases = [Path('/content/drive/MyDrive')]
shared_root = Path('/content/drive/Shared drives')
if shared_root.exists():
    bases.append(shared_root)

patterns = [
    '*seoul*modified*',
    '*seoul*bike*mod*',
    '*modified*seoul*',
    '*bike*sharing*mod*',
]

hits = []
for base in bases:
    for pat in patterns:
        hits.extend(p for p in base.rglob(pat) if p.suffix.lower() in {'.csv', '.xlsx'})

seen = set(); unique = []
for p in hits:
    if str(p) not in seen:
        seen.add(str(p)); unique.append(p)

print(f'Encontrados: {len(unique)}')
for i, p in enumerate(unique[:200]):
    print(f'[{i}] {p}')


In [ ]:
#@title 🧩 Usar el mismo archivo para original y modificado
from pathlib import Path

# Ruta del único archivo que tienes (ajusta si tu carpeta cambia)
p_org = Path("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/Copia de seoul_bike_sharing_original.csv")
p_mod = p_org  # Usa el mismo archivo temporalmente

assert p_org.exists(), f"❌ No existe: {p_org}"
print("✅ Archivos listos (ambos apuntan al original):")
print("Original:", p_org)
print("Modified:", p_mod)


In [ ]:
#@title 1️⃣ Leer el dataset original
import pandas as pd

df_org = pd.read_csv(p_org)
df_mod = pd.read_csv(p_mod)

print("✅ Archivos cargados correctamente")
print("Dimensiones del dataset:", df_org.shape)
df_org.head()


In [ ]:
#@title 2️⃣ Crear versión modificada en la misma carpeta
import pandas as pd

df_mod = df_org.copy()

# Ejemplo de modificación: eliminar 10 primeras filas
df_mod = df_mod.iloc[10:, :]

output_path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/seoul_bike_sharing_modified.csv"
df_mod.to_csv(output_path, index=False)

print("✅ Archivo modificado creado en Drive:", output_path)


In [ ]:
#@title 1️⃣ Actualizar ruta al archivo modificado
from pathlib import Path

p_mod = Path("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/seoul_bike_sharing_modified.csv")

assert p_mod.exists(), f"❌ No existe el archivo modificado: {p_mod}"
print("✅ Archivo modificado detectado correctamente:")
print(p_mod)


In [ ]:
#@title 2️⃣ Leer ambos archivos (original y modificado)
import pandas as pd

df_org = pd.read_csv(p_org)
df_mod = pd.read_csv(p_mod)

print("✅ Archivos cargados correctamente")
print(f"Original:  {df_org.shape} filas x columnas")
print(f"Modificado: {df_mod.shape} filas x columnas")

display(df_org.head(2))
display(df_mod.head(2))


In [ ]:
#@title 3️⃣ Comparar diferencias básicas
diff_rows = len(df_org) - len(df_mod)
print(f"El dataset modificado tiene {diff_rows} filas menos que el original.")


## 3) Carga (principal=modified, referencia=original)

In [ ]:
#@title Cargar
org = pd.read_csv(p_org); mod = pd.read_csv(p_mod)
org.head(2), mod.head(2)

In [ ]:
mod

In [ ]:
org

## 4) Utilidades

In [ ]:
#@title Funciones
def normalize_cols(df):
    df = df.copy(); df.rename(columns={c: c.strip().replace("\\xa0"," ").replace("  "," ").strip() for c in df.columns}, inplace=True); return df
def add_parsed_date(df):
    df = df.copy(); d=[c for c in df.columns if "date" in c.lower()]; df["__Date"]=pd.to_datetime(df[d[0]], errors="coerce", dayfirst=True) if d else pd.NaT
    h=[c for c in df.columns if "hour" in c.lower()]; df["__Hour"]=pd.to_numeric(df[h[0]], errors="coerce") if h else np.nan; return df
def guess_target(cols):
    for c in cols:
        if "rented" in c.lower() and "count" in c.lower(): return c
    return None
def clean_df(df, target_col):
    df=df.copy()
    for c in df.select_dtypes(include=["object"]).columns:
        df[c]=df[c].astype(str).str.strip().replace({"nan": np.nan})
    for c in df.columns:
        if c!=target_col and df[c].dtype==object:
            num=pd.to_numeric(df[c].str.replace(",","").str.replace("%",""), errors="coerce")
            if num.notna().sum()>=0.5*len(df): df[c]=num
    for c in df.select_dtypes(include=[np.number]).columns:
        q1,q99=df[c].quantile(0.01), df[c].quantile(0.99)
        if pd.notna(q1) and pd.notna(q99) and q99>q1: df[c]=df[c].clip(q1,q99)
    return df
def detect_categoricals(df, target_col):
    cats=list(df.select_dtypes(include=["object","category","bool"]).columns)
    for c in df.select_dtypes(include=[np.number]).columns:
        if df[c].nunique(dropna=True)<=20 and c!=target_col: cats.append(c)
    return sorted([c for c in set(cats) if not c.startswith("__")])


## 5) Objetivo y EDA rápida

In [ ]:
#@title 🎯 Objetivo y EDA

# Normalización de columnas y fechas
org = normalize_cols(org)
mod = normalize_cols(mod)
org = add_parsed_date(org)
mod = add_parsed_date(mod)

# Detección automática de columna objetivo
target_col = guess_target(mod.columns) or guess_target(org.columns)
if target_col is None:
    n = mod.select_dtypes(include=np.number).columns
    target_col = n[0] if len(n) > 0 else mod.columns[0]

print("🔹 Columna objetivo detectada:", target_col)

# --- Función de resumen (overview) ---
def overview(df, name):
    info = pd.DataFrame({
        "col": df.columns,
        "dtype": df.dtypes.astype(str),
        "n_missing": df.isna().sum().values,
        "pct_missing": (100 * df.isna().sum() / len(df)).round(2).values,
        "n_unique": [df[c].nunique(dropna=True) for c in df.columns]
    })
    display(info.sort_values("pct_missing", ascending=False).style.set_caption(name))

# Mostrar resúmenes de ambos conjuntos
overview(mod, "Modified (pre-limpieza)")
overview(org, "Original (referencia)")

# --- Análisis de la variable objetivo ---
plt.figure(figsize=(8, 5))

# Limpieza básica de la columna objetivo (quita $, comas, espacios)
s = mod[target_col].astype(str).str.replace(r"[,\s\$\€]", "", regex=True)
num = pd.to_numeric(s, errors="coerce")  # convierte a numérico, fuerza NaN si hay texto

print(f"Total de filas: {len(num)}")
print(f"Valores válidos: {num.notna().sum()}  |  Nulos o no convertibles: {num.isna().sum()}")

# Graficar distribución
num.dropna().astype(float).plot(kind="hist", bins=30)
plt.title(f"Distribución del objetivo (Modified, pre-limpieza): {target_col}")
plt.xlabel(target_col)
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()


In [ ]:
# Usaremos df_mod como "mod" para que tu celda funcione
mod = df_mod.copy()

# Define el target del dataset de Seoul Bike
target_col = 'Rented Bike Count'  # <- este sí existe en tu CSV

print("Columnas disponibles en 'mod':")
print(list(mod.columns))


In [ ]:
#@title 🔍 Análisis forense de una columna categórica (listo para correr)
# Cambia aquí la columna a analizar:
COL = 'Seasons'  # <-- edita: 'Seasons' | 'Holiday' | 'Functioning Day' | etc.

import numpy as np, pandas as pd, re, warnings
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

warnings.filterwarnings("ignore")

# ===== 0) Preparación de datos =====
# Detecta df_mod / df_org y define 'mod' de trabajo
if 'df_mod' in globals():
    mod = df_mod.copy()
elif 'df_org' in globals():
    mod = df_org.copy()
else:
    raise RuntimeError("No encuentro 'df_mod' ni 'df_org'. Carga primero tu dataset con pandas.")

# Target típico del dataset de Seoul Bike (ajusta si usas otro):
target_col = 'Rented Bike Count'
if target_col not in mod.columns:
    raise KeyError(f"No encuentro la columna target '{target_col}' en el DataFrame.")

# Sugerencias si la columna elegida no existe
if COL not in mod.columns:
    obj_cols = [c for c in mod.columns if mod[c].dtype == 'object']
    mixed_type_cols = [c for c in mod.columns if mod[c].map(type).nunique() > 1]
    msg = [
        f"❌ La columna '{COL}' no existe.",
        "👉 Columnas disponibles (primeras 25): " + ", ".join(list(mod.columns)[:25]),
        "💡 Categóricas (object): " + (", ".join(obj_cols) if obj_cols else "(ninguna)"),
        "🧪 Posibles 'mixtas' (tipos mezclados): " + (", ".join(mixed_type_cols) if mixed_type_cols else "(ninguna)"),
    ]
    raise KeyError("\n".join(msg))

print(f"=== Análisis de: `{COL}` ===")

# ===== 1) Perfil básico =====
s = mod[COL]
print("dtype:", s.dtype)
n = len(s)
n_missing = s.isna().sum()
n_unique = s.nunique(dropna=True)
print(f"filas={n:,} | n_missing={n_missing:,} ({100*n_missing/n:.2f}%) | n_unique={n_unique:,} ({100*n_unique/n:.2f}% del total)")

types_counts = Counter(type(x).__name__ for x in s)
print("Tipos de valores observados:", dict(types_counts))

top_vals = pd.Series(s.astype(str)).value_counts(dropna=False).head(12)
print("\nTop 12 valores (crudos):")
print(top_vals)

# ===== 2) Convertibilidad a numérico =====
print("\n=== 2) ¿Se puede convertir a numérico? ===")
sn = (
    s.astype(str)
     .str.strip()
     .str.replace(r"[,\s%]", "", regex=True)
     .str.replace(r"[^\-\.\dEe+]", "", regex=True)
)
num = pd.to_numeric(sn, errors='coerce')
pct_numeric = 100 * num.notna().mean()
print(f"% convertibles a número: {pct_numeric:.2f}%")
if num.notna().sum() > 0:
    print("Resumen (solo convertibles):")
    print(num.dropna().astype(float).describe())

# ===== 3) Asociación con el target =====
print("\n=== 3) Asociación con el target ===")
y = pd.to_numeric(mod[target_col], errors='coerce')
mask = y.notna()
y_valid = y[mask]

if num.notna().sum() > 0:
    corr = np.corrcoef(num[mask].fillna(num[mask].median()), y_valid)[0,1]
    print(f"Correlación numérica con {target_col}: {corr:.4f}")
else:
    print("No hay suficientes valores numéricos para correlación directa.")

if s.nunique(dropna=True) <= 50:
    grp = pd.DataFrame({"col": s[mask], "y": y_valid}).dropna().groupby("col")["y"].mean().sort_values(ascending=False)
    print("\nMedia del target por categoría (top 10):")
    print(grp.head(10))

# ===== 4) ¿Predice demasiado bien por sí sola? =====
print("\n=== 4) ¿Predice demasiado bien por sí sola? ===")
from sklearn.ensemble import RandomForestRegressor

df_tmp = mod[[COL, target_col]].copy()
df_tmp[target_col] = pd.to_numeric(df_tmp[target_col], errors='coerce')
df_tmp = df_tmp.dropna(subset=[target_col]).copy()
X_one = df_tmp[[COL]]
y_one = df_tmp[target_col].astype(float)

preproc = ColumnTransformer([
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), [COL])
])
model_one = Pipeline(steps=[
    ('enc', preproc),
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42))
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores_r2 = cross_val_score(model_one, X_one, y_one, scoring='r2', cv=cv)
scores_rmse = -cross_val_score(model_one, X_one, y_one, scoring='neg_root_mean_squared_error', cv=cv)

print(f"CV R² (solo `{COL}`):  mean={scores_r2.mean():.4f} ±{scores_r2.std():.4f}")
print(f"CV RMSE (solo `{COL}`): mean={scores_rmse.mean():.2f} ±{scores_rmse.std():.2f}")

# ===== 5) Comparación con y sin la columna =====
print("\n=== 5) Comparación con y sin la columna ===")
mod_numish = mod.copy()
for c in mod_numish.select_dtypes('object').columns:
    coerced = pd.to_numeric(mod_numish[c].astype(str).str.replace(r"[,\s%]", "", regex=True), errors='coerce')
    if coerced.notna().mean() >= 0.6:
        mod_numish[c] = coerced

A = mod_numish.dropna(subset=[target_col]).copy()
A_y = pd.to_numeric(A[target_col], errors='coerce')
A = A.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
A = A.loc[A_y.notna()]
A_y = A_y.loc[A_y.notna()].astype(float)

X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(A, A_y, test_size=0.2, random_state=42)
rf_A = RandomForestRegressor(n_estimators=200, random_state=42)
rf_A.fit(X_train_A, y_train_A)
r2_A = r2_score(y_test_A, rf_A.predict(X_test_A))
print(f"R² (CON `{COL}`): {r2_A:.4f}")

B = mod_numish.drop(columns=[COL], errors='ignore')
B = B.dropna(subset=[target_col]).copy()
B_y = pd.to_numeric(B[target_col], errors='coerce')
B = B.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
B = B.loc[B_y.notna()]
B_y = B_y.loc[B_y.notna()].astype(float)

X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(B, B_y, test_size=0.2, random_state=42)
rf_B = RandomForestRegressor(n_estimators=200, random_state=42)
rf_B.fit(X_train_B, y_train_B)
r2_B = r2_score(y_test_B, rf_B.predict(X_test_B))
print(f"R² (SIN `{COL}`): {r2_B:.4f}")

# ===== 6) Señales empíricas + 7) Recomendación =====
print("\n=== 6) Señales empíricas ===")
flags = []
if n_unique / n > 0.9:
    flags.append("cardinalidad_altisima")
if len(types_counts) > 1:
    flags.append("tipos_mezclados")
if scores_r2.mean() >= 0.2:
    flags.append("predice_demasiado_bien_sola")
if (r2_A - r2_B > 0.05):
    flags.append("impacto_excesivo_en_modelo")

print("Flags activadas:", flags if flags else "ninguna")

print("\n=== 7) Recomendación preliminar ===")
if flags:
    print(f"→ Sugerencia: revisar/posible eliminar `{COL}` del modelado. Motivos:")
    if "cardinalidad_altisima" in flags: print("  - Cardinalidad ~única por fila (parece ID o hash).")
    if "tipos_mezclados" in flags: print("  - Mezcla de tipos (strings/números/NaN).")
    if "predice_demasiado_bien_sola" in flags: print("  - Predice mucho por sí sola; validar CV temporal para evitar sobreajuste estacional.")
    if "impacto_excesivo_en_modelo" in flags: print("  - Aporta ganancia anómala al R²; validar importancia y fuga potencial.")
else:
    print(f"→ Mantener `{COL}` con cautela; no hay señales fuertes de fuga. Validar con CV y splits temporales.")



In [ ]:
#@title 🔎 Benchmark de TODAS las columnas categóricas (resumen y flags)
import pandas as pd, numpy as np
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from pathlib import Path

# 0) Preparación
if 'df_mod' in globals():
    base_df = df_mod.copy()
elif 'df_org' in globals():
    base_df = df_org.copy()
else:
    raise RuntimeError("No encuentro df_mod ni df_org. Carga el dataset primero.")

target_col = 'Rented Bike Count'
assert target_col in base_df.columns, f"No está {target_col} en el DataFrame."

# Copia para coerción numérica ligera
def coerce_numish(df):
    df = df.copy()
    for c in df.select_dtypes('object').columns:
        cc = pd.to_numeric(df[c].astype(str).str.replace(r"[,\s%]", "", regex=True), errors='coerce')
        if cc.notna().mean() >= 0.6:
            df[c] = cc
    return df

mod = base_df.copy()
mod_numish = coerce_numish(mod)

# Lista de candidatas categóricas
cat_cols = [c for c in mod.columns if mod[c].dtype == 'object' and c != target_col]
results = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for COL in cat_cols:
    s = mod[COL]
    n = len(s)
    n_missing = s.isna().sum()
    n_unique = s.nunique(dropna=True)
    types_counts = Counter(type(x).__name__ for x in s)

    # --- Modelo SOLO con COL ---
    df_tmp = mod[[COL, target_col]].copy()
    df_tmp[target_col] = pd.to_numeric(df_tmp[target_col], errors='coerce')
    df_tmp = df_tmp.dropna(subset=[target_col]).copy()
    X_one = df_tmp[[COL]]
    y_one = df_tmp[target_col].astype(float)

    preproc = ColumnTransformer([
        ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), [COL])
    ])
    model_one = Pipeline(steps=[
        ('enc', preproc),
        ('rf', RandomForestRegressor(n_estimators=200, random_state=42))
    ])

    try:
        scores_r2 = cross_val_score(model_one, X_one, y_one, scoring='r2', cv=kf)
        scores_rmse = -cross_val_score(model_one, X_one, y_one, scoring='neg_root_mean_squared_error', cv=kf)
        r2_solo = scores_r2.mean()
        rmse_solo = scores_rmse.mean()
    except Exception as e:
        r2_solo, rmse_solo = np.nan, np.nan

    # --- Modelo global CON la columna ---
    A = mod_numish.dropna(subset=[target_col]).copy()
    A_y = pd.to_numeric(A[target_col], errors='coerce')
    A = A.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
    A = A.loc[A_y.notna()]
    A_y = A_y.loc[A_y.notna()].astype(float)

    if A.shape[1] >= 1:
        X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(A, A_y, test_size=0.2, random_state=42)
        rf_A = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_A, y_train_A)
        r2_A = r2_score(y_test_A, rf_A.predict(X_test_A))
    else:
        r2_A = np.nan

    # --- Modelo global SIN la columna (si es numérica no afecta; si es object sí la quita) ---
    B = mod_numish.drop(columns=[COL], errors='ignore')
    B = B.dropna(subset=[target_col]).copy()
    B_y = pd.to_numeric(B[target_col], errors='coerce')
    B = B.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
    B = B.loc[B_y.notna()]
    B_y = B_y.loc[B_y.notna()].astype(float)

    if B.shape[1] >= 1:
        X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(B, B_y, test_size=0.2, random_state=42)
        rf_B = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train_B, y_train_B)
        r2_B = r2_score(y_test_B, rf_B.predict(X_test_B))
    else:
        r2_B = np.nan

    delta_r2 = (r2_A - r2_B) if (pd.notna(r2_A) and pd.notna(r2_B)) else np.nan

    # --- Flags (mismas reglas que tu celda) ---
    flags = []
    if n_unique / n > 0.9:
        flags.append("cardinalidad_altisima")
    if len(types_counts) > 1:
        flags.append("tipos_mezclados")
    if pd.notna(r2_solo) and r2_solo >= 0.20:
        flags.append("predice_demasiado_bien_sola")
    if pd.notna(delta_r2) and delta_r2 > 0.05:
        flags.append("impacto_excesivo_en_modelo")

    results.append({
        "col": COL,
        "dtype": str(s.dtype),
        "filas": n,
        "n_missing": int(n_missing),
        "pct_missing": round(100*n_missing/n, 2),
        "n_unique": int(n_unique),
        "cardinalidad": round(n_unique/n, 4),
        "tipos_observados": dict(types_counts),
        "CV_R2_sola": round(r2_solo, 4) if pd.notna(r2_solo) else np.nan,
        "CV_RMSE_sola": round(rmse_solo, 2) if pd.notna(rmse_solo) else np.nan,
        "R2_con_col": round(r2_A, 4) if pd.notna(r2_A) else np.nan,
        "R2_sin_col": round(r2_B, 4) if pd.notna(r2_B) else np.nan,
        "delta_R2": round(delta_r2, 4) if pd.notna(delta_r2) else np.nan,
        "flags": ", ".join(flags) if flags else "",
    })

summary = pd.DataFrame(results).sort_values(
    by=["delta_R2", "CV_R2_sola", "cardinalidad"], ascending=[False, False, True]
).reset_index(drop=True)

print("✅ Resumen de columnas categóricas analizadas:")
display(summary.head(10))

# Guardar a Drive (misma carpeta del dataset)
out_path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/summary_categoricas.csv"
summary.to_csv(out_path, index=False)
print("💾 Guardado en:", out_path)


In [ ]:
#@title 🕒 Parseo robusto de fechas y variables temporales
import pandas as pd
import numpy as np

assert 'mod' in globals(), "No encuentro el DataFrame 'mod'. Asegura que ya cargaste df_org/df_mod y definiste 'mod'."

col = 'Date'
assert col in mod.columns, f"No existe la columna '{col}' en mod."

# Normaliza separadores y limpia espacios
s = mod[col].astype(str).str.strip().str.replace(r'[\.\-]', '/', regex=True)

# 1) Intento principal: formatos mixtos con día primero
dt = pd.to_datetime(s, format='mixed', dayfirst=True, errors='coerce')

# 2) Reintentos para lo que quede sin parsear (algunos datasets traen %Y/%m/%d)
mask = dt.isna()
if mask.any():
    dt.loc[mask] = pd.to_datetime(s[mask], format='%Y/%m/%d', errors='coerce')
mask = dt.isna()
if mask.any():
    dt.loc[mask] = pd.to_datetime(s[mask], format='%d/%m/%Y', errors='coerce')
mask = dt.isna()

# Diagnóstico rápido
bad = int(mask.sum())
print(f"✅ Parseadas: {len(dt)-bad:,}  |  ❌ Sin parsear: {bad:,}")
if bad:
    # Muestra ejemplos problemáticos
    print("Ejemplos sin parsear:", s[mask].unique()[:5])

# Si todo salió bien, seguimos
assert dt.notna().mean() > 0.98, "Demasiadas fechas sin parsear. Revisa el tipo de valores impresos arriba."

# Asigna y crea variables temporales
mod[col] = dt
mod['year']       = dt.dt.year
mod['month']      = dt.dt.month
mod['dayofweek']  = dt.dt.dayofweek  # 0=Lunes
mod['is_weekend'] = mod['dayofweek'].isin([5,6]).astype(int)
mod['day']        = dt.dt.day

print("Variables creadas:", ['year','month','dayofweek','is_weekend','day'])
mod.head(3)


In [ ]:
#@title 🧹 Eliminar 'Date' cruda y preparar X, y
target_col = "Rented Bike Count"
mod = mod.drop(columns=["Date"], errors="ignore")
X = mod.drop(columns=[target_col], errors="ignore")
y = mod[target_col]
print("Shape X:", X.shape, "| Shape y:", y.shape)


In [ ]:
#@title 📆 Validación temporal con preprocesamiento (OneHot + Imputación)
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Asegura que ya tienes 'mod' con las columnas temporales creadas
assert 'mod' in globals(), "No encuentro el DataFrame 'mod'. Corre primero la celda que crea year/month/dayofweek/etc."
target_col = "Rented Bike Count"
assert target_col in mod.columns, f"No existe '{target_col}' en mod."

# 1) Orden temporal
mod_sorted = mod.sort_values(['year','month','day']).reset_index(drop=True)

# 2) X, y y selección de tipos
X_sorted = mod_sorted.drop(columns=[target_col])
y_sorted = mod_sorted[target_col]

cat_cols = X_sorted.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_sorted.columns.difference(cat_cols).tolist()

print("Categóricas:", cat_cols)
print("Numéricas:", num_cols[:10], " ...")

# 3) Preprocesamiento: imputación + one-hot para categóricas
preprocess = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), num_cols),
        ('cat', Pipeline(steps=[
            ('imp', SimpleImputer(strategy='most_frequent')),
            ('oh', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_cols)
    ],
    remainder='drop'
)

# 4) Modelo en pipeline
model = Pipeline(steps=[
    ('prep', preprocess),
    ('rf', RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1))
])

# 5) Validación temporal (sin fuga)
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, X_sorted, y_sorted, cv=tscv, scoring='r2')
print("✅ R² promedio (TimeSeriesSplit):", round(np.mean(scores), 4))
print("   Std:", round(np.std(scores), 4))

# 6) Entrenamiento final con split temporal 80/20 y residuales
split_idx = int(len(mod_sorted)*0.8)
X_train, y_train = X_sorted.iloc[:split_idx], y_sorted.iloc[:split_idx]
X_test,  y_test  = X_sorted.iloc[split_idx:], y_sorted.iloc[split_idx:]

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# --- Calcular métricas manualmente (compatible con versiones viejas)
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

# RMSE = raíz cuadrada del MSE manualmente
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"\n✅ R² (test temporal): {r2:.4f}")
print(f"📉 RMSE: {rmse:.2f}")

# --- Gráfica de residuales
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_pred - y_test, alpha=0.5)
plt.axhline(0, linestyle='--', color='red')
plt.xlabel("Rented Bike Count (real)")
plt.ylabel("Residual (pred - real)")
plt.title("Residuales (split temporal 80/20)")
plt.show()



In [ ]:
#@title 🌟 Importancia de las variables (feature importance)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Extraer importancias desde el modelo dentro del pipeline
rf_model = model.named_steps['rf']
encoder = model.named_steps['prep']

# Obtener nombres de columnas transformadas
cat_cols = encoder.transformers_[1][2]
num_cols = encoder.transformers_[0][2]
oh_names = list(encoder.named_transformers_['cat'].named_steps['oh'].get_feature_names_out(cat_cols))
all_features = num_cols + oh_names

# Crear DataFrame con importancias
feat_imp = pd.DataFrame({
    'Variable': all_features,
    'Importancia': rf_model.feature_importances_
}).sort_values('Importancia', ascending=False)

# Mostrar top 15
print("📊 Top 15 variables más importantes:")
display(feat_imp.head(15))

# Graficar
plt.figure(figsize=(8,5))
plt.barh(feat_imp.head(15)['Variable'][::-1], feat_imp.head(15)['Importancia'][::-1])
plt.title("Importancia de las variables (Random Forest)")
plt.xlabel("Importancia")
plt.show()


## 6) Limpieza SOLO en modified

In [ ]:
#@title Limpieza y guardado
mod_clean = clean_df(mod, target_col)
mod_clean[target_col] = pd.to_numeric(mod_clean[target_col], errors="coerce")

# Eliminar columna problemática si existe
if "mixed_type_col" in mod_clean.columns:
    mod_clean.drop(columns=["mixed_type_col"], inplace=True)
    print("Columna 'mixed_type_col' eliminada.")

# Guardar solo el dataset limpio modificado
out_dir = Path(p_mod).parent if 'p_mod' in globals() else Path('.')
mod_clean_path = out_dir / 'cleaned_modified.csv'
mod_clean.to_csv(mod_clean_path, index=False)

print("Guardado:", mod_clean_path)
print("Tamaño final ->", mod_clean.shape)


In [ ]:
import pandas as pd

mod = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_modified.csv")
print("Columnas disponibles:")
print(list(mod.columns))


## 7) ¿Se conserva la distribución? Comparación vs original

In [ ]:
#@title 🧩 Feature Engineering (funciona con o sin columna de fecha)
import pandas as pd
import numpy as np

in_path  = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_modified.csv"
out_path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched.csv"

mod = pd.read_csv(in_path)

# Normaliza encabezados
mod.columns = mod.columns.str.strip().str.lower().str.replace(' ', '_')

print("Columnas detectadas:", mod.columns.tolist())

# === 1) Obtener/crear columna fecha ===
fecha_col = None
for c in mod.columns:
    if "date" in c:  # por si viene 'date', 'datetime', 'dteday', etc.
        fecha_col = c
        break

if fecha_col is not None:
    # Parseo directo
    dt = pd.to_datetime(mod[fecha_col], dayfirst=True, errors='coerce')
else:
    # Intento reconstruir con year/month/day si existen
    if all(col in mod.columns for col in ["year", "month", "day"]):
        dt = pd.to_datetime(
            dict(year=mod["year"], month=mod["month"], day=mod["day"]),
            errors="coerce"
        )
        fecha_col = "fecha_reconstruida"
        mod[fecha_col] = dt
        print("✅ Fecha reconstruida desde year/month/day → columna:", fecha_col)
    else:
        dt = None
        print("⚠️ No hay columna de fecha ni (year, month, day). Se omiten features de calendario.")

# === 2) Features temporales ===
if dt is not None:
    mod["dayofyear"]  = dt.dt.dayofyear
    # week can be UInt32 in pandas; conviértela a int
    mod["weekofyear"] = dt.dt.isocalendar().week.astype("int64")
    mod["quarter"]    = dt.dt.quarter
    # Si no existe dayofweek/is_weekend los creamos
    if "dayofweek" not in mod.columns:
        mod["dayofweek"] = dt.dt.dayofweek
    if "is_weekend" not in mod.columns:
        mod["is_weekend"] = (mod["dayofweek"] >= 5).astype(int)

# === 3) Codificación cíclica de la hora (si existe 'hour') ===
if "hour" in mod.columns:
    mod["hour_sin"] = np.sin(2 * np.pi * mod["hour"] / 24)
    mod["hour_cos"] = np.cos(2 * np.pi * mod["hour"] / 24)
else:
    print("⚠️ No existe columna 'hour'; se omiten hour_sin/hour_cos.")

# === 4) Interacciones y términos no lineales (si existen numéricas) ===
if "temperature(°c)" in mod.columns and "humidity(%)" in mod.columns:
    mod["temp_humidity"] = mod["temperature(°c)"] * mod["humidity(%)"]
if "solar_radiation_(mj/m2)" in mod.columns and "wind_speed_(m/s)" in mod.columns:
    mod["solar_wind"] = mod["solar_radiation_(mj/m2)"] * mod["wind_speed_(m/s)"]
if "temperature(°c)" in mod.columns:
    mod["temp^2"] = mod["temperature(°c)"] ** 2
if "humidity(%)" in mod.columns:
    mod["humidity^2"] = mod["humidity(%)"] ** 2

# === 5) Guardar ===
mod.to_csv(out_path, index=False)
print("\n✅ Dataset enriquecido guardado.")
print("📂", out_path)
print("📏 shape:", mod.shape)


In [ ]:
#@title 🚀 Reentrenamiento del modelo con el dataset enriquecido
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

# === 1) Cargar el dataset enriquecido ===
path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched.csv"
mod = pd.read_csv(path)

print("✅ Dataset cargado:", mod.shape)
print("Columnas disponibles:", mod.columns.tolist())

# === 2) Identificar target y variables ===
target_col = "rented_bike_count"

# Eliminar columnas no útiles o duplicadas del target
X = mod.drop(columns=[target_col], errors="ignore")
y = mod[target_col]

# === 3) Codificar variables categóricas ===
cat_cols = X.select_dtypes(include=["object", "bool"]).columns
if len(cat_cols) > 0:
    print("Categóricas detectadas:", cat_cols.tolist())
    enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    X[cat_cols] = enc.fit_transform(X[cat_cols])
else:
    print("No se detectaron variables categóricas.")

# === 4) Definir modelo y validación temporal ===
tscv = TimeSeriesSplit(n_splits=5)
r2_scores, rmse_scores = [], []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))


    r2_scores.append(r2)
    rmse_scores.append(rmse)

print("\n📈 Resultados de validación temporal:")
print(f"R² promedio: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"RMSE promedio: {np.mean(rmse_scores):.2f}")

# === 5) Entrenar modelo final con todo el dataset ===
model.fit(X, y)
print("\n✅ Modelo final entrenado con todo el dataset.")

# === 6) Gráfico de residuales ===
y_pred_all = model.predict(X)
residuals = y_pred_all - y

plt.figure(figsize=(8,5))
plt.scatter(y, residuals, alpha=0.4)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Rented Bike Count (real)")
plt.ylabel("Residual (pred - real)")
plt.title("Residuals (modelo final)")
plt.show()

# === 7) Importancia de variables ===
importances = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model.feature_importances_
}).sort_values(by="Importancia", ascending=False)

print("\n🏆 Top 15 variables más importantes:")
display(importances.head(15))

plt.figure(figsize=(8,5))
plt.barh(importances.head(15)["Variable"], importances.head(15)["Importancia"])
plt.gca().invert_yaxis()
plt.title("Importancia de las variables (Random Forest)")
plt.xlabel("Importancia")
plt.show()


In [ ]:
#@title 🧩 Construir timestamp y crear *lags* y promedios móviles (no-leak)
import pandas as pd
import numpy as np

# 1) Carga el enriquecido
path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched.csv"
df = pd.read_csv(path)

# 2) Normaliza encabezados
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# 3) Reconstruye datetime si existe 'fecha_reconstruida' + 'hour'
if "fecha_reconstruida" in df.columns and "hour" in df.columns:
    dt = pd.to_datetime(df["fecha_reconstruida"], errors="coerce")
    ts = dt + pd.to_timedelta(df["hour"], unit="h")
    df["ts"] = ts
else:
    # si no existe, intenta 'year','month','day','hour'
    if all(c in df.columns for c in ["year","month","day","hour"]):
        dt = pd.to_datetime(dict(year=df["year"], month=df["month"], day=df["day"]), errors="coerce")
        df["ts"] = dt + pd.to_timedelta(df["hour"], unit="h")
    else:
        raise RuntimeError("No pude construir un timestamp. Necesito fecha_reconstruida+hour o year+month+day+hour.")

# 4) Orden temporal
df = df.sort_values("ts").reset_index(drop=True)

# 5) Crear lags SIN fuga (shift desplaza hacia atrás)
y = df["rented_bike_count"]
df["lag_1h"]   = y.shift(1)
df["lag_24h"]  = y.shift(24)       # mismo día anterior, misma hora
df["lag_168h"] = y.shift(24*7)     # una semana antes, misma hora

# 6) Promedios móviles usando solo pasado (shift(1) evita mirar el futuro)
df["roll_mean_24h"]  = y.shift(1).rolling(24).mean()
df["roll_mean_168h"] = y.shift(1).rolling(24*7).mean()
df["roll_max_24h"]   = y.shift(1).rolling(24).max()
df["roll_min_24h"]   = y.shift(1).rolling(24).min()

# 7) Elimina las filas al inicio que quedan con NaN por lags
min_warmup = 24*7   # 1 semana
df = df.iloc[min_warmup:].copy()

# 8) Guardar versión con lags
out_path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags.csv"
df.to_csv(out_path, index=False)
print("✅ Dataset con lags guardado:", out_path)
print("shape:", df.shape)


In [ ]:
#@title 🚀 Entrenar con TimeSeriesSplit usando lags (evita usar la fecha cruda)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor

# 1) Carga
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

target = "rented_bike_count"

# 2) ARMAR X (sin usar la fecha cruda). Dejamos 'ts' solo para ordenar, NO como feature.
drop_cols = [target, "ts", "fecha_reconstruida"]  # sacamos la fecha categórica
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
y = df[target]

# 3) Detectar tipos
cat_cols = X.select_dtypes(include=["object","bool"]).columns.tolist()
num_cols = X.columns.difference(cat_cols).tolist()

print("Categóricas:", cat_cols)
print("Numéricas (ejemplo):", num_cols[:10], "...")

# 4) Preprocesamiento
prep = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ],
    remainder="drop"
)

# 5) Modelo principal (HGBR suele ir mejor que RF con muchas numéricas)
# Si HGBR te da problemas en tu versión, cambia 'reg' por RandomForestRegressor(...)
reg = HistGradientBoostingRegressor(
    learning_rate=0.08,
    max_depth=10,
    max_iter=400,
    l2_regularization=0.0,
    random_state=42
)

pipe = Pipeline(steps=[("prep", prep), ("reg", reg)])

# 6) Validación temporal
tscv = TimeSeriesSplit(n_splits=5)
r2s, rmses = [], []

for fold, (tr, te) in enumerate(tscv.split(X), 1):
    Xtr, Xte = X.iloc[tr], X.iloc[te]
    ytr, yte = y.iloc[tr], y.iloc[te]

    pipe.fit(Xtr, ytr)
    pred = pipe.predict(Xte)

    r2 = r2_score(yte, pred)
    rmse = np.sqrt(mean_squared_error(yte, pred))
    r2s.append(r2); rmses.append(rmse)
    print(f"Fold {fold}: R²={r2:.4f}  RMSE={rmse:.2f}")

print("\n📈 CV temporal → R² mean:", round(np.mean(r2s),4), "±", round(np.std(r2s),4))
print("📉 CV temporal → RMSE mean:", round(np.mean(rmses),2))

# 7) Fit final y residuales (80/20 temporal)
split_idx = int(len(X)*0.8)
pipe.fit(X.iloc[:split_idx], y.iloc[:split_idx])
y_pred = pipe.predict(X.iloc[split_idx:])

r2 = r2_score(y.iloc[split_idx:], y_pred)
rmse = np.sqrt(mean_squared_error(y.iloc[split_idx:], y_pred))
print(f"\n✅ Test temporal 80/20: R²={r2:.4f}  RMSE={rmse:.2f}")

plt.figure(figsize=(8,5))
plt.scatter(y.iloc[split_idx:], y_pred - y.iloc[split_idx:], alpha=0.4)
plt.axhline(0, color="red", ls="--")
plt.xlabel("Rented Bike Count (real)")
plt.ylabel("Residual (pred - real)")
plt.title("Residuales (con lags)")
plt.show()


In [ ]:
#@title 🔧 GridSearch para optimizar el modelo (HGBR o RF)
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score, make_scorer

# Cargar dataset enriquecido con lags
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

target = "rented_bike_count"
drop_cols = [target, "ts", "fecha_reconstruida"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
y = df[target]

# Identificar tipos
cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()
num_cols = X.columns.difference(cat_cols).tolist()

# Preprocesador
prep = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ]
)


In [ ]:
#@title 🚀 Configurar búsqueda de hiperparámetros
reg = HistGradientBoostingRegressor(random_state=42)

pipe = Pipeline(steps=[("prep", prep), ("reg", reg)])

param_grid = {
    "reg__learning_rate": [0.03, 0.05, 0.08, 0.1],
    "reg__max_depth": [8, 10, 12, None],
    "reg__max_iter": [300, 400, 500],
    "reg__l2_regularization": [0.0, 0.1, 0.5]
}

tscv = TimeSeriesSplit(n_splits=5)

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=tscv,
    scoring=make_scorer(r2_score),
    n_jobs=-1,
    verbose=2
)

grid.fit(X, y)


In [ ]:
#@title 📈 Resultados del mejor modelo
print("🏆 Mejor combinación encontrada:")
print(grid.best_params_)
print("\n🔹 Mejor R² promedio en CV temporal:", round(grid.best_score_, 4))

best_model = grid.best_estimator_

# Entrena final con todo el dataset
best_model.fit(X, y)

# Guardar modelo (opcional)
import joblib
joblib.dump(best_model, "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/best_model.pkl")

print("\n✅ Modelo optimizado guardado como best_model.pkl")


In [ ]:
# 📊 Top combinaciones del GridSearch
cv = pd.DataFrame(grid.cv_results_)
cols = ["mean_test_score","std_test_score","rank_test_score"] + [c for c in cv.columns if c.startswith("param_")]
top = cv.sort_values("rank_test_score").head(15)[cols]
top.rename(columns={"mean_test_score":"R2_CV_mean","std_test_score":"R2_CV_std","rank_test_score":"rank"}, inplace=True)
display(top)


In [ ]:
#@title 🏁 Evaluación final 80/20 temporal + export de predicciones
import pandas as pd, numpy as np, joblib, matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# 1) Carga dataset con lags (ordenado por ts) y rearmar X,y como en el grid
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.sort_values("ts").reset_index(drop=True)

target = "rented_bike_count"
drop_cols = [target, "ts", "fecha_reconstruida"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
y = df[target]

# 2) Cargar el mejor modelo ya entrenado por GridSearch
best_model = joblib.load("/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/best_model.pkl")

# 3) Split temporal 80/20, fit y métricas
split_idx = int(len(X)*0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

r2  = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"✅ Test temporal 80/20 — R²: {r2:.4f}   RMSE: {rmse:.2f}")

# 4) Residuales
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_pred - y_test, alpha=0.4)
plt.axhline(0, color="red", ls="--")
plt.xlabel("Rented Bike Count (real)")
plt.ylabel("Residual (pred - real)")
plt.title("Residuales del mejor modelo (80/20 temporal)")
plt.show()

# 5) Export de predicciones
preds = pd.DataFrame({
    "ts": df.loc[split_idx:, "ts"].values if "ts" in df.columns else np.arange(len(y_test)),
    "y_real": y_test.values,
    "y_pred": y_pred,
    "residual": y_pred - y_test.values
})
out_csv = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/test_predictions_best_model.csv"
preds.to_csv(out_csv, index=False)
print("💾 Predicciones guardadas en:", out_csv)


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_fino = {
    "reg__learning_rate": [0.07, 0.09, 0.1, 0.11],
    "reg__max_depth": [10, 12, 14],
    "reg__max_iter": [400, 500, 600],
    "reg__l2_regularization": [0.3, 0.5, 0.7]
}

grid_fino = GridSearchCV(
    grid.best_estimator_,
    param_grid=param_grid_fino,
    cv=tscv,
    scoring=make_scorer(r2_score),
    n_jobs=-1,
    verbose=2
)

grid_fino.fit(X, y)
print("Nuevo mejor R² promedio:", round(grid_fino.best_score_, 4))
print("Mejores parámetros:", grid_fino.best_params_)


In [ ]:
#@title 🧩 Features suaves: rolling/lag/interaction (sin fuga) + guardado
import pandas as pd
import numpy as np

in_path  = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags.csv"
out_path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags_plus.csv"

df = pd.read_csv(in_path)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Asegura orden temporal por 'ts' si existe
if "ts" in df.columns:
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")
    df = df.sort_values("ts").reset_index(drop=True)
else:
    # Si no hay ts, intenta reconstruir para mantener orden
    if all(c in df.columns for c in ["year","month","day","hour"]):
        base_dt = pd.to_datetime(dict(year=df["year"], month=df["month"], day=df["day"]), errors="coerce")
        df["ts"] = base_dt + pd.to_timedelta(df["hour"], unit="h")
        df = df.sort_values("ts").reset_index(drop=True)

# --- 1) Rolling de 6h en temperatura (shift(1) evita ver el futuro)
if "temperature(°c)" in df.columns:
    df["temp_roll_6h"] = df["temperature(°c)"].shift(1).rolling(6).mean()

# --- 2) Rolling de 3h en lluvia
if "rainfall(mm)" in df.columns:
    df["rain_roll_3h"] = df["rainfall(mm)"].shift(1).rolling(3).mean()

# --- 3) Lag 24h del target (si no existiera o quieres actualizarlo)
if "rented_bike_count" in df.columns:
    df["lag_24h"] = df["rented_bike_count"].shift(24)

# --- 4) Interacción temperatura x radiación solar
if set(["temperature(°c)", "solar_radiation_(mj/m2)"]).issubset(df.columns):
    df["temp_x_solar"] = df["temperature(°c)"] * df["solar_radiation_(mj/m2)"]

# Warmup: elimina filas con NaN por lags/rollings (tomamos 24h)
warmup = 24
df = df.iloc[warmup:].copy()

df.to_csv(out_path, index=False)
print("✅ Guardado con nuevas features en:", out_path)
print("📏 shape:", df.shape)
print("🆕 Nuevas columnas:", [c for c in df.columns if c in ["temp_roll_6h","rain_roll_3h","lag_24h","temp_x_solar"]])


In [ ]:
#@title 📊 Reporte visual: métricas y gráficos (80/20 temporal)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.inspection import permutation_importance

path = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE/cleaned_enriched_lags_plus.csv"
df = pd.read_csv(path)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Orden temporal
if "ts" in df.columns:
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")
    df = df.sort_values("ts").reset_index(drop=True)

target = "rented_bike_count"
drop_cols = [target, "ts", "fecha_reconstruida"]  # no usamos la fecha cruda como feature
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
y = df[target]

# Categóricas / numéricas
cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()
num_cols = X.columns.difference(cat_cols).tolist()

prep = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ],
    remainder="drop"
)

# Modelo (ajusta si quieres los params encontrados en GridSearch)
reg = HistGradientBoostingRegressor(
    learning_rate=0.1, max_depth=12, max_iter=500,
    l2_regularization=0.5, random_state=42
)
pipe = Pipeline(steps=[("prep", prep), ("reg", reg)])

# Split 80/20 temporal
split_idx = int(len(X)*0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

r2  = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = np.mean(np.abs(y_test - y_pred))
print(f"✅ Test temporal 80/20 → R²: {r2:.4f} | RMSE: {rmse:.2f} | MAE: {mae:.2f}")

# === 1) Importancia de variables (robusta) ===
from sklearn.inspection import permutation_importance

try:
    # Preferido: PI sobre las columnas ORIGINALES (mismas longitudes)
    pi = permutation_importance(
        pipe, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )
    feat_names = X_test.columns.tolist()                      # <-- clave
    imp_df = pd.DataFrame(
        {"Variable": feat_names, "Importance": pi.importances_mean}
    )
except ValueError:
    # Fallback: usar feature_importances_ del modelo en el ESPACIO TRANSFORMADO
    reg_est = pipe.named_steps["reg"]
    if not hasattr(reg_est, "feature_importances_"):
        raise RuntimeError(
            "No se pudo calcular importancias: el estimador no expone 'feature_importances_'."
        )

    # Nombres de columnas después del preprocesamiento
    try:
        feat_names = pipe.named_steps["prep"].get_feature_names_out().tolist()
    except Exception:
        oh = None
        if cat_cols:
            oh = pipe.named_steps["prep"].named_transformers_["cat"].named_steps.get("oh", None)
        oh_names = oh.get_feature_names_out(cat_cols).tolist() if oh is not None else []
        feat_names = list(num_cols) + oh_names

    imp_df = pd.DataFrame(
        {"Variable": feat_names, "Importance": reg_est.feature_importances_}
    )

# Ordenar y mostrar
imp_df = imp_df.sort_values("Importance", ascending=False)

print("\n🏆 Top 15 variables más importantes:")
display(imp_df.head(15))

plt.figure(figsize=(9,5))
plt.barh(imp_df.head(15)["Variable"][::-1], imp_df.head(15)["Importance"][::-1])
plt.title("Importancia de variables")
plt.xlabel("Importancia")
plt.tight_layout()
plt.show()


In [ ]:
#@title 🧪 Stats lado a lado + KS test (robusto: detecta target y reconstruye mod_clean si falta)
import re
import pandas as pd
from scipy.stats import ks_2samp

# ------- Utilidades robustas -------
def normalize_cols_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # minúsculas, sin espacios dobles, reemplaza espacios por _
    df.columns = (df.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(r"\s+", " ", regex=True)
                  .str.replace(" ", "_"))
    return df

def detect_target_column(cols) -> str:
    """
    Devuelve el nombre 'real' del target entre las columnas,
    buscando variantes comunes de 'Rented Bike Count'.
    """
    candidates_exact = [
        "rented_bike_count",
        "rentedbikecount",
        "rented_bikes_count"
    ]
    # primero intenta coincidencia exacta (normalizada)
    for c in candidates_exact:
        if c in cols:
            return c
    # luego intenta heurística por tokens
    pat = re.compile(r"(rented).*?(bike).*?(count)", re.I)
    for c in cols:
        if pat.search(c.replace("_", " ")):
            return c
    # si no encuentra, intenta coincidencia parcial
    for c in cols:
        if "rented" in c and "bike" in c and "count" in c:
            return c
    raise KeyError("No pude detectar la columna target (rented_bike_count). Revisa nombres de columnas.")

# ------- 1) Asegurar org_clean -------
if 'p_org' not in globals():
    raise RuntimeError("No encuentro 'p_org'. Ejecuta la celda de montaje/carga de rutas (la que define p_org).")

org = pd.read_csv(p_org)
org = normalize_cols_df(org)

# Si tienes funciones propias de limpieza, aplícalas (opcionales)
if 'clean_df' in globals():
    # Primero detecta target en org para que clean_df pueda usarlo si lo requiere
    try:
        target_col_org = detect_target_column(list(org.columns))
    except KeyError:
        target_col_org = None
    org_clean = clean_df(org, target_col_org) if target_col_org else clean_df(org, None)
    org_clean = normalize_cols_df(org_clean)
else:
    # fallback simple si no existe clean_df
    org_clean = org.copy()
    # elimina columnas completamente vacías
    org_clean = org_clean.dropna(axis=1, how='all')

# Detecta target definitivo en org_clean
target_col = detect_target_column(list(org_clean.columns))

# convertir target a numérico por consistencia
org_clean[target_col] = pd.to_numeric(org_clean[target_col], errors="coerce")

# ------- 2) Asegurar mod_clean -------
if 'mod_clean' not in globals():
    # intenta reconstruir desde tu CSV limpio
    from pathlib import Path
    base_dir = "/content/drive/MyDrive/Colab Notebooks/SEXTO TRIMESTRE"
    # prioriza el archivo más completo si existe
    candidates = [
        "cleaned_modified.csv",
        "cleaned_enriched.csv",
        "cleaned_enriched_lags.csv",
        "cleaned_enriched_lags_plus.csv"
    ]
    chosen = None
    for name in candidates:
        p = Path(base_dir) / name
        if p.exists():
            chosen = str(p)
            break
    if chosen is None:
        raise RuntimeError("No encuentro 'mod_clean' ni archivos limpios en Drive (cleaned_modified.csv / enriched...).")
    mod_clean = pd.read_csv(chosen)
    mod_clean = normalize_cols_df(mod_clean)
else:
    mod_clean = normalize_cols_df(mod_clean)

# Alinea el nombre del target también en mod_clean (por si difiere)
try:
    target_col_mod = detect_target_column(list(mod_clean.columns))
except KeyError:
    target_col_mod = target_col  # usa el mismo nombre si coincide tras normalizar

# asegurar tipo numérico en el target de mod
if target_col_mod in mod_clean.columns:
    mod_clean[target_col_mod] = pd.to_numeric(mod_clean[target_col_mod], errors="coerce")

# ------- 3) KS test en columnas comunes numéricas -------
common = [c for c in mod_clean.columns if c in org_clean.columns]

rows = []
for c in common:
    # saltar columnas claramente no comparables (ids, timestamps, texto duro)
    if c in {target_col, target_col_mod, "ts", "fecha_reconstruida"}:
        pass  # puedes comentar esta línea si quieres comparar también el target
    a = pd.to_numeric(mod_clean[c], errors="coerce").dropna()
    b = pd.to_numeric(org_clean[c], errors="coerce").dropna()
    if len(a) > 50 and len(b) > 50:
        stat, p = ks_2samp(a, b)
        rows.append({
            "col": c,
            "mean_mod": a.mean(),
            "std_mod": a.std(),
            "p50_mod": a.median(),
            "mean_org": b.mean(),
            "std_org": b.std(),
            "p50_org": b.median(),
            "ks_stat": stat,
            "ks_pvalue": p
        })

cmp_df = pd.DataFrame(rows).sort_values("ks_stat", ascending=False)

if len(cmp_df) > 0:
    display(cmp_df.style.set_caption(
        "Comparación (modified limpio vs. original limpio) — KS p>0.05 ≈ sin cambio estadístico fuerte"
    ))
else:
    print("⚠️ No hubo suficientes columnas numéricas comunes con datos en ambos datasets para comparar con KS.")
print(f"Target usado para org_clean: '{target_col}'  |  para mod_clean: '{target_col_mod}'")


## 8) Baseline ML (solo modified limpio)

In [ ]:
#@title 8 BIS) Baseline ML

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.inspection import permutation_importance

# ------------------ Parámetros editables ------------------
TEST_SIZE = 0.15
VAL_SIZE  = 0.15
RANDOM_STATE = 42
DO_STANDARDIZE = False          # árboles no lo necesitan
N_ESTIMATORS = 500              # un poco más robusto
MAX_DEPTH = 20                  # profundidad moderada
MIN_SAMPLES_LEAF = 2            # hoja mínima para reducir overfitting
MAX_FEATURES = 'sqrt'           # estrategia común en RF
N_PERMUT = 10
TOPK_IMP = 15
USE_SEASONS_AS_ORDINAL = False  # Seasons como nominal por ciclo anual
# ----------------------------------------------------------

# --- Chequeos previos ---
if 'mod_clean' not in globals():
    raise RuntimeError("No se encontró 'mod_clean'. Ejecuta la celda de limpieza primero.")
if 'target_col' not in globals() or target_col not in mod_clean.columns:
    raise KeyError("No se encontró 'target_col' o no está en mod_clean.columns")

# Copia de trabajo + salvaguarda por si mixed sigue ahí
df = mod_clean.copy()
if "mixed_type_col" in df.columns:
    df = df.drop(columns=["mixed_type_col"])

# Target numérico + drop filas sin target
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
df = df.dropna(subset=[target_col]).copy()

# ---------- Feature engineering cíclico para Hour ----------
# Si existe Hour (o __Hour), convierte a num y crea sin/cos
hour_candidates = [c for c in ['Hour', '__Hour'] if c in df.columns]
if hour_candidates:
    hcol = hour_candidates[0]
    df[hcol] = pd.to_numeric(df[hcol], errors='coerce')
    # reemplazar outliers/NaN razonablemente
    df[hcol] = df[hcol].clip(0, 23)
    df['Hour_sin'] = np.sin(2*np.pi*df[hcol]/24.0)
    df['Hour_cos'] = np.cos(2*np.pi*df[hcol]/24.0)
    # (Opcional) si quieres evitar doble conteo, puedes eliminar la original:
    # df.drop(columns=[hcol], inplace=True)

y = df[target_col].astype(float)
X = df.drop(columns=[target_col])

# Detectar tipos base
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_all  = [c for c in X.columns if c not in num_cols]

# --- Definir ordinales vs nominales ---
ordinal_specs_all = []

# Seasons -> por defecto NOMINAL (USE_SEASONS_AS_ORDINAL=False)
if 'Seasons' in X.columns and USE_SEASONS_AS_ORDINAL:
    ordinal_specs_all.append(("Seasons", ["Winter", "Spring", "Summer", "Autumn"]))

# Functioning Day: binaria ordenada (si existe)
if 'Functioning Day' in X.columns:
    ordinal_specs_all.append(("Functioning Day", ["No", "Yes"]))

ordinal_cols   = [name for (name, order) in ordinal_specs_all if name in X.columns]
ordinal_orders = [order for (name, order) in ordinal_specs_all if name in X.columns]
cat_nominal = [c for c in cat_all if c not in ordinal_cols]

print(f"Dataset final: {df.shape} | Numéricas: {len(num_cols)} | "
      f"Cat nominales: {len(cat_nominal)} | Cat ordinales: {len(ordinal_cols)} | Target n: {len(y)}")

# ---------- Estratificación (binning si regresión) ----------
is_classification = False
if y.nunique() <= 20:
    is_classification = True

if is_classification:
    stratify_labels = y
else:
    n_bins = min(10, int(np.sqrt(len(y))))
    try:
        stratify_labels = pd.qcut(y, q=n_bins, duplicates='drop')
    except Exception:
        stratify_labels = pd.cut(y, bins=n_bins)

# ---------- Split: test y luego val ----------
X_rest, X_test, y_rest, y_test, strat_rest, strat_test = train_test_split(
    X, y, stratify_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
val_frac_of_rest = VAL_SIZE / (1.0 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_rest, y_rest, test_size=val_frac_of_rest, random_state=RANDOM_STATE, stratify=strat_rest
)

print(f"Split sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

# ---------- Preprocesamiento ----------
num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler() if DO_STANDARDIZE else "passthrough")
])

# OneHot compatible con distintas versiones de sklearn
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn < 1.2

cat_nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe)
])

cat_ordinal_transformer = None
if ordinal_cols:
    cat_ordinal_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder(
            categories=ordinal_orders,
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ))
    ])

transformers = []
if num_cols:
    transformers.append(("num", num_transformer, num_cols))
if cat_nominal:
    transformers.append(("cat_nom", cat_nominal_transformer, cat_nominal))
if ordinal_cols and cat_ordinal_transformer is not None:
    transformers.append(("cat_ord", cat_ordinal_transformer, ordinal_cols))

preproc = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

# ---------- Modelo ----------
if is_classification:
    model = RandomForestClassifier(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=RANDOM_STATE,
        n_jobs=-1
    )
else:
    model = RandomForestRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=RANDOM_STATE,
        min_samples_leaf=MIN_SAMPLES_LEAF, max_features=MAX_FEATURES, n_jobs=-1
    )

pipe = Pipeline(steps=[("preproc", preproc), ("model", model)])

# Entrenar
pipe.fit(X_train, y_train)

# Predecir
y_val_pred = pipe.predict(X_val)
y_test_pred = pipe.predict(X_test)

# ---------- Métricas ----------
if is_classification:
    print("\nVALIDACIÓN - Clasificación:")
    print(classification_report(y_val, y_val_pred))
    print("TEST - Clasificación:")
    print(classification_report(y_test, y_test_pred))
    print("Test accuracy:", accuracy_score(y_test, y_test_pred))
else:
    print("\nVALIDACIÓN - Regresión:")
    print("MAE:", mean_absolute_error(y_val, y_val_pred))
    print("MSE:", mean_squared_error(y_val, y_val_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))
    print("R2:", r2_score(y_val, y_val_pred))
    print("\nTEST - Regresión (evaluación final):")
    print("MAE:", mean_absolute_error(y_test, y_test_pred))
    print("MSE:", mean_squared_error(y_test, y_test_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
    print("R2:", r2_score(y_test, y_test_pred))

# ---------- Importancia por permutación (alineada a columnas de ENTRADA) ----------
try:
    input_feature_names = []
    if num_cols:      input_feature_names += list(num_cols)
    if cat_nominal:   input_feature_names += list(cat_nominal)
    if ordinal_cols:  input_feature_names += list(ordinal_cols)

    result = permutation_importance(
        pipe, X_val, y_val, n_repeats=N_PERMUT, random_state=RANDOM_STATE, n_jobs=-1
    )
    imp = pd.Series(result.importances_mean, index=input_feature_names).sort_values(ascending=False)
    print(f"\nTop {min(TOPK_IMP, len(imp))} features por permutación (nivel columnas de entrada):")
    display(imp.head(TOPK_IMP))
except Exception as e:
    print("No se pudo calcular la importancia por permutación:", e)

print("\nNotas:")
print("- Seasons se trata como NOMINAL por defecto (switch USE_SEASONS_AS_ORDINAL=False).")
print("- Hour ahora incluye Hour_sin/Hour_cos; se puede eliminar la columna Hour original si se dejo.")
print("- Árboles sin escalado; RF con min_samples_leaf y max_features para mejor generalización.")
print("- Guarda el pipeline con joblib.dump(pipe, 'best_model.joblib').")


In [ ]:
#@title 9) XGBoost + features temporales/derivadas + LOG1P + MAPE + permutación

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

# ------------------ Parámetros editables ------------------
TEST_SIZE = 0.15
VAL_SIZE  = 0.15
RANDOM_STATE = 42

# Preprocesamiento
DO_STANDARDIZE_NUM = False      # boosting no lo necesita
USE_SEASONS_AS_ORDINAL = False  # Seasons como nominal (ciclo anual)
USE_HOUR_SIN_COS = True         # mantener Hour_sin / Hour_cos si se generaron antes

# Modelo (ajusta aquí)
TRY_XGBOOST = True              # si no está instalado, fallback a HistGradientBoosting
XGB_PARAMS = dict(
    n_estimators=700,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    tree_method="hist",
    n_jobs=-1
)
HGB_PARAMS = dict(
    max_depth=None, learning_rate=0.05, max_iter=600, random_state=RANDOM_STATE
)

# Target
LOG1P_TARGET = True            # activar para estabilizar varianza del target
N_PERMUT = 10
TOPK_IMP = 15
# ----------------------------------------------------------

# --- Chequeos y base ---
if 'mod_clean' not in globals():
    raise RuntimeError("No se encontró 'mod_clean'. Ejecuta la celda de limpieza primero.")
if 'target_col' not in globals() or target_col not in mod_clean.columns:
    raise KeyError("No se encontró 'target_col' o no está en mod_clean.columns")

df = mod_clean.copy()
if "mixed_type_col" in df.columns:
    df = df.drop(columns=["mixed_type_col"])

# --- Features temporales desde Date ---
def ensure_datetime(s):
    return pd.to_datetime(s, errors='coerce')

if 'Date' in df.columns:
    dt = ensure_datetime(df['Date'])
    df['Year']    = dt.dt.year
    df['Month']   = dt.dt.month
    df['Weekday'] = dt.dt.dayofweek  # 0=Lunes ... 6=Domingo
    df['is_weekend'] = (df['Weekday'] >= 5).astype(int)
    # Codificación cíclica Month/Weekday
    df['Month_sin']   = np.sin(2*np.pi*(df['Month']-1)/12.0)
    df['Month_cos']   = np.cos(2*np.pi*(df['Month']-1)/12.0)
    df['Weekday_sin'] = np.sin(2*np.pi*df['Weekday']/7.0)
    df['Weekday_cos'] = np.cos(2*np.pi*df['Weekday']/7.0)

# Si existen Hour/__Hour y no se crearon sin/cos antes, activarlo aquí
if USE_HOUR_SIN_COS:
    for hcol in ['Hour', '__Hour']:
        if hcol in df.columns:
            df[hcol] = pd.to_numeric(df[hcol], errors='coerce').clip(0, 23)
            if 'Hour_sin' not in df.columns:
                df['Hour_sin'] = np.sin(2*np.pi*df[hcol]/24.0)
            if 'Hour_cos' not in df.columns:
                df['Hour_cos'] = np.cos(2*np.pi*df[hcol]/24.0)
            # (Opcional) evita duplicidad si dejas sin/cos
            # df.drop(columns=[hcol], inplace=True)
            break

# --- Features DERIVADAS útiles ---
# Hora pico
if 'Hour' in df.columns:
    df['is_rush_hour'] = df['Hour'].isin([7,8,9,17,18,19]).astype(int)
elif '__Hour' in df.columns:
    df['is_rush_hour'] = df['__Hour'].isin([7,8,9,17,18,19]).astype(int)

# Comodidad percibida aprox.
if {'Temperature(°C)', 'Humidity(%)'}.issubset(df.columns):
    df['comfort_index'] = df['Temperature(°C)'] - df['Humidity(%)']/5.0

# Disconfort por viento
if {'Wind speed (m/s)', 'Humidity(%)'}.issubset(df.columns):
    df['wind_discomfort'] = df['Wind speed (m/s)'] * df['Humidity(%)']

# Feriado o fin de semana (normaliza Holiday a binaria)
if 'is_weekend' in df.columns and 'Holiday' in df.columns:
    h = df['Holiday'].astype(str).str.lower().isin(['holiday','yes','1','true'])
    df['is_holiday_or_weekend'] = ((df['is_weekend'] == 1) | h).astype(int)

# --- Target y split base ---
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
df = df.dropna(subset=[target_col]).copy()

if LOG1P_TARGET:
    y_raw = df[target_col].astype(float)
    y = np.log1p(y_raw)
else:
    y = df[target_col].astype(float)

X = df.drop(columns=[target_col])

# Detectar tipos
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_all  = [c for c in X.columns if c not in num_cols]

# Definir ordinales vs nominales
ordinal_specs_all = []
if 'Seasons' in X.columns and USE_SEASONS_AS_ORDINAL:
    ordinal_specs_all.append(("Seasons", ["Winter", "Spring", "Summer", "Autumn"]))
if 'Functioning Day' in X.columns:
    ordinal_specs_all.append(("Functioning Day", ["No", "Yes"]))

ordinal_cols   = [name for (name, order) in ordinal_specs_all if name in X.columns]
ordinal_orders = [order for (name, order) in ordinal_specs_all if name in X.columns]
cat_nominal = [c for c in cat_all if c not in ordinal_cols]

print(f"Dataset final: {df.shape} | Numéricas: {len(num_cols)} | "
      f"Cat nominales: {len(cat_nominal)} | Cat ordinales: {len(ordinal_cols)} | Target n: {len(y)}")

# Estratificación por bins (regresión)
n_bins = min(10, int(np.sqrt(len(y))))
try:
    stratify_labels = pd.qcut(y, q=n_bins, duplicates='drop')
except Exception:
    stratify_labels = pd.cut(y, bins=n_bins)

X_rest, X_test, y_rest, y_test, strat_rest, strat_test = train_test_split(
    X, y, stratify_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
val_frac_of_rest = VAL_SIZE / (1.0 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_rest, y_rest, test_size=val_frac_of_rest, random_state=RANDOM_STATE, stratify=strat_rest
)
print(f"Split sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

# --- Preprocesamiento ---
num_steps = [("imputer", SimpleImputer(strategy="median"))]
if DO_STANDARDIZE_NUM:
    num_steps.append(("scaler", StandardScaler()))
num_transformer = Pipeline(steps=num_steps)

# OneHot compatible versiones
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn < 1.2

cat_nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe)
])

cat_ordinal_transformer = None
if ordinal_cols:
    cat_ordinal_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder(
            categories=ordinal_orders,
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ))
    ])

transformers = []
if num_cols:
    transformers.append(("num", num_transformer, num_cols))
if cat_nominal:
    transformers.append(("cat_nom", cat_nominal_transformer, cat_nominal))
if ordinal_cols and cat_ordinal_transformer is not None:
    transformers.append(("cat_ord", cat_ordinal_transformer, ordinal_cols))

preproc = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

# --- Modelo: XGBoost o fallback a HistGradientBoosting ---
model = None
used_xgb = False
if TRY_XGBOOST:
    try:
        from xgboost import XGBRegressor
        model = XGBRegressor(**XGB_PARAMS)
        used_xgb = True
        print("Modelo: XGBRegressor")
    except Exception as e:
        print("XGBoost no disponible o falló la importación; usando HistGradientBoostingRegressor.", e)

if model is None:
    from sklearn.ensemble import HistGradientBoostingRegressor
    model = HistGradientBoostingRegressor(**HGB_PARAMS)
    print("Modelo: HistGradientBoostingRegressor")

pipe = Pipeline(steps=[("preproc", preproc), ("model", model)])

# ---------- (Opcional) Búsqueda automática rápida ----------
# from sklearn.model_selection import RandomizedSearchCV
# param_grid = {
#     "model__n_estimators": [400, 600, 800],
#     "model__learning_rate": [0.03, 0.05, 0.1],
#     "model__max_depth": [4, 6, 8],
#     "model__subsample": [0.7, 0.8, 0.9],
#     "model__colsample_bytree": [0.7, 0.8, 1.0],
#     "model__reg_lambda": [0.5, 1.0, 2.0]
# }
# search = RandomizedSearchCV(pipe, param_distributions=param_grid, n_iter=12,
#                             scoring="r2", cv=3, n_jobs=-1, random_state=42, verbose=1)
# search.fit(X_train, y_train)
# pipe = search.best_estimator_
# print("Mejores parámetros:", search.best_params_)

# Entrenar
pipe.fit(X_train, y_train)

# Predecir
y_val_pred = pipe.predict(X_val)
y_test_pred = pipe.predict(X_test)

# Invertir transformación del target para métricas en escala original
def inv_target(z):
    return np.expm1(z) if LOG1P_TARGET else z

y_val_eval = inv_target(y_val)
y_test_eval = inv_target(y_test)
y_val_pred_eval = inv_target(y_val_pred)
y_test_pred_eval = inv_target(y_test_pred)

# Métricas + MAPE
def report_split(name, y_true, y_hat):
    mae = mean_absolute_error(y_true, y_hat)
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_hat)

    # MAPE evitando división por cero
    m = (np.abs((y_true - y_hat)[y_true != 0] / y_true[y_true != 0])).mean() * 100 if np.any(y_true != 0) else np.nan

    print(f"{name}:")
    print(f"  MAE:  {mae:.3f}")
    print(f"  MSE:  {mse:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R2:   {r2:.6f}")
    print(f"  MAPE: {m:.2f}%")

print("\nVALIDACIÓN - Regresión:")
report_split("VAL", y_val_eval, y_val_pred_eval)

print("\nTEST - Regresión (evaluación final):")
report_split("TEST", y_test_eval, y_test_pred_eval)

# --- Importancia por permutación (nivel columnas de entrada) ---
try:
    input_feature_names = []
    if num_cols:      input_feature_names += list(num_cols)
    if cat_nominal:   input_feature_names += list(cat_nominal)
    if ordinal_cols:  input_feature_names += list(ordinal_cols)

    result = permutation_importance(
        pipe, X_val, y_val, n_repeats=N_PERMUT, random_state=RANDOM_STATE, n_jobs=-1
    )
    imp = pd.Series(result.importances_mean, index=input_feature_names).sort_values(ascending=False)
    print(f"\nTop {min(TOPK_IMP, len(imp))} features por permutación (entrada):")
    display(imp.head(TOPK_IMP))
except Exception as e:
    print("No se pudo calcular la importancia por permutación:", e)

print("\nNotas:")
print("- Añadidas: is_rush_hour, comfort_index, wind_discomfort, is_holiday_or_weekend.")
print(f"- Modelo usado: {'XGBRegressor' if used_xgb else 'HistGradientBoostingRegressor'}; LOG1P_TARGET={LOG1P_TARGET}.")
print("- Ajusta XGB_PARAMS o activa la búsqueda automática (RandomizedSearchCV) para subir R².")


In [ ]:
#@title 10) Diagnóstico visual mejorado (ticks limpios, sanitización, zoom y bins)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# Build DataFrame de evaluación
# =========================
df_eval = pd.DataFrame({"y_real": y_test_eval, "y_pred": y_test_pred_eval})
df_eval["residual"] = df_eval["y_real"] - df_eval["y_pred"]

if "X_test" in locals():
    df_plot = pd.concat([df_eval.reset_index(drop=True), X_test.reset_index(drop=True)], axis=1)
else:
    df_plot = df_eval.copy()

# =========================
# Helpers
# =========================
def yclip(series, q=0.99):
    """Devuelve un (min, max) simétrico por cuantil absoluto para 'zoom'."""
    lim = series.abs().quantile(q)
    lim = float(lim) if np.isfinite(lim) and lim > 0 else float(series.abs().max() or 1.0)
    return (-lim, lim)

def safe_num(s, lo=None, hi=None, round_to_int=False, allow_na_int=True, clip=True):
    """Convierte a numérico, limpia inf, recorta y opcionalmente redondea/castea a entero con NA."""
    x = pd.to_numeric(s, errors="coerce").replace([np.inf, -np.inf], np.nan)
    if clip and (lo is not None or hi is not None):
        x = x.clip(lower=lo, upper=hi)
    if round_to_int:
        x = x.round()
        if allow_na_int:
            x = x.astype("Int64")  # entero que acepta NA
        else:
            x = x.fillna(0).astype(int)
    return x

# =========================
# 1) Pred vs Real
# =========================
plt.figure(figsize=(7,6))
sns.scatterplot(data=df_plot, x="y_real", y="y_pred", s=15, alpha=0.6)
minv = float(df_plot[["y_real","y_pred"]].min().min())
maxv = float(df_plot[["y_real","y_pred"]].max().max())
plt.plot([minv, maxv], [minv, maxv], ls="--", color="red", label="y = x")
plt.legend(loc="upper left")
plt.title("Predicción vs Real (Test)")
plt.xlabel("Real"); plt.ylabel("Predicho")
plt.tight_layout(); plt.show()

# =========================
# 2) Distribución de residuales (completa y con zoom)
# =========================
plt.figure(figsize=(8,4))
sns.histplot(df_plot["residual"], bins=60, kde=True)
plt.title("Distribución de residuales (y_real - y_pred)")
plt.xlabel("Residual")
plt.tight_layout(); plt.show()

low, high = yclip(df_plot["residual"], 0.995)
plt.figure(figsize=(8,4))
sns.histplot(df_plot["residual"].clip(low, high), bins=60, kde=True)
plt.title("Distribución de residuales (zoom 99.5%)")
plt.xlabel("Residual (recortado)")
plt.tight_layout(); plt.show()

# =========================
# 3) Residuales vs Predicción (zoom 99%)
# =========================
plt.figure(figsize=(7,4))
ylim = yclip(df_plot["residual"], 0.99)
sns.scatterplot(data=df_plot, x="y_pred", y=df_plot["residual"].clip(*ylim), s=10, alpha=0.6)
plt.axhline(0, ls="--", color="red")
plt.ylim(ylim)
plt.title("Residuales vs Predicción (zoom 99%)")
plt.xlabel("Predicción"); plt.ylabel("Residual")
plt.tight_layout(); plt.show()

# =========================
# 4) Por HORA: boxplot + MAE (con sanitización)
# =========================
if "Hour" in df_plot.columns or "__Hour" in df_plot.columns:
    hcol = "Hour" if "Hour" in df_plot.columns else "__Hour"
    tmp = df_plot.copy()
    # Sanitizar a [0,23], entero con NA permitido
    tmp[hcol] = safe_num(tmp[hcol], lo=0, hi=23, round_to_int=True, allow_na_int=True, clip=True)
    tmp["Hour_str"] = tmp[hcol].astype(str).fillna("NA")

    ylim = yclip(tmp["residual"], 0.99)
    plt.figure(figsize=(12,4))
    sns.boxplot(data=tmp, x="Hour_str", y=tmp["residual"].clip(*ylim))
    plt.ylim(ylim); plt.title("Residuales por Hour (zoom 99%)")
    plt.xlabel("Hour"); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    # MAE por hora (excluye NA)
    mae_hour = (
        tmp[tmp["Hour_str"] != "NA"]
        .groupby(hcol)["residual"]
        .apply(lambda s: s.abs().mean())
        .sort_index()
    )
    plt.figure(figsize=(12,3))
    mae_hour.plot(kind="bar")
    plt.title("MAE por Hour")
    plt.xlabel("Hour"); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

# =========================
# 5) Por WEEKDAY: boxplot + MAE (con sanitización)
# =========================
if "Weekday" in df_plot.columns:
    tmp = df_plot.copy()
    tmp["Weekday"] = safe_num(tmp["Weekday"], lo=0, hi=6, round_to_int=True, allow_na_int=True, clip=True)
    day_labels = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
    # Mapear con NA seguro
    mapper = dict(enumerate(day_labels))
    tmp["Weekday_lbl"] = tmp["Weekday"].map(mapper).astype(object)
    tmp["Weekday_lbl"] = tmp["Weekday_lbl"].fillna("NA")

    ylim = yclip(tmp["residual"], 0.99)
    plt.figure(figsize=(9,4))
    sns.boxplot(data=tmp, x="Weekday_lbl", y=tmp["residual"].clip(*ylim))
    plt.ylim(ylim); plt.title("Residuales por Weekday (zoom 99%)")
    plt.xlabel("Weekday"); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    # MAE por weekday (excluye NA)
    mae_wd = (
        tmp[tmp["Weekday_lbl"] != "NA"]
        .groupby("Weekday_lbl")["residual"]
        .apply(lambda s: s.abs().mean())
        .reindex(day_labels)  # asegurar orden Lun..Dom
    )
    plt.figure(figsize=(9,3))
    mae_wd.plot(kind="bar")
    plt.title("MAE por Weekday")
    plt.xlabel("Weekday"); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

# =========================
# 6) Temperatura / Humedad: agrupar en BINS (qcut) y MAE por bin
# =========================
def boxplot_by_bins(df_in, col, bins=10, label=None):
    if col not in df_in.columns:
        return
    tmp = df_in[[col, "residual"]].dropna().copy()
    if tmp.empty:
        return
    # Bins por cuantiles; si hay duplicados de borde, qcut los maneja
    try:
        tmp["bin"] = pd.qcut(tmp[col], q=bins, duplicates="drop")
    except ValueError:
        # Si hay muy pocos valores distintos
        tmp["bin"] = pd.cut(tmp[col], bins=min(bins, 5))
    ylim = yclip(tmp["residual"], 0.99)

    plt.figure(figsize=(12,4))
    sns.boxplot(data=tmp, x="bin", y=tmp["residual"].clip(*ylim))
    plt.xticks(rotation=45, ha="right")
    plt.ylim(ylim); plt.title(f"Residuales por {label or col} (bins, zoom 99%)")
    plt.xlabel(label or col); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    mae_bin = tmp.groupby("bin")["residual"].apply(lambda s: s.abs().mean())
    plt.figure(figsize=(12,3))
    mae_bin.plot(kind="bar")
    plt.xticks(rotation=45, ha="right")
    plt.title(f"MAE por {label or col} (bins)")
    plt.xlabel(label or col); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

boxplot_by_bins(df_plot, "Temperature(°C)", bins=12, label="Temperature(°C)")
boxplot_by_bins(df_plot, "Humidity(%)",     bins=12, label="Humidity(%)")

# =========================
# 7) Correlaciones de residuales con numéricas
# =========================
num_cols_eval = df_plot.select_dtypes(include=[np.number]).columns
if "residual" in num_cols_eval:
    corr_resid = df_plot[num_cols_eval].corr()["residual"].sort_values(ascending=False)
    print("🔹 Correlaciones de residuales con variables numéricas (positivas = sobreestimación):")
    display(corr_resid.head(15))

# =========================
# 8) Top outliers para inspección
# =========================
topk = 10
print(f"\n🔎 Top {topk} residuales absolutos (para inspección):")
cols_show = ["y_real","y_pred","residual"]
for c in ["Date","Hour","__Hour","Weekday","Temperature(°C)","Humidity(%)","Rainfall(mm)","Snowfall (cm)","Holiday","is_weekend","is_holiday_or_weekend"]:
    if c in df_plot.columns:
        cols_show.append(c)

top_df = (
    df_plot.reindex(columns=[c for c in cols_show if c in df_plot.columns])
    .iloc[np.argsort(-df_plot['residual'].abs())[:topk]]
)
display(top_df)
